In [18]:
import router_security
import json
import time 
import requests
import os

In [16]:
with open('local_ip_record.json','r') as file_input:
    record = json.load(file_input)
    print(record)
    file_input.close()

record['10.0.0.1'] = {'time':(time.time()),'brand':'cisco','level':5}

with open('local_ip_record.json','w') as file:
    print('json', (json.dumps(record)))
    file.write((json.dumps(record)))
    file.close()
        


{'10.0.0.1': {'time': '1686899152.353969', 'brand': 'cisco', 'level': 5}}
json {"10.0.0.1": {"time": 1686899168.4513128, "brand": "cisco", "level": 5}}


In [29]:
myip = requests.get('https://api.bigdatacloud.net/data/client-ip').json()['ipString']
print(myip)

42.77.97.93


In [114]:
import nmap 

IP = '10.159.67.65'
RESULT = {'snmp':None,'os':None,'isp':None,'location':None,'time':None}

def detect_snmp_or_os():
    nm = nmap.PortScanner()
    try:
        nm.scan(hosts=IP, arguments='-sU -sV -p 161 --script=snmp-info',timeout=10)
        for info in nm[IP]['udp'][161]['script']['snmp-info'].split('\n '):
            if 'enterprise: ' in info:
                brand = info[info.find('enterprise: ')+len('enterprise: '):]
        # RESULT['snmp'] = {'name':nm[IP]['udp'][161]['product'],'vendor':brand}
        return brand

    except nmap.nmap.PortScannerTimeout:
        print('SNMP scanning timeout. Doing OS detect...')
        nm.scan(hosts=IP, arguments='-O')
        # RESULT['os'] = {'name':nm[IP]['osmatch'][0]['name'],'vendor':nm[IP]['osmatch'][0]['osclass'][0]['vendor']}
        return nm[IP]['osmatch'][0]['osclass'][0]['vendor']
    except KeyError:
        print('SNMP closed. Doing OS detect...')
        nm.scan(hosts=IP, arguments='-O')
        # RESULT['os'] = {'name':nm[IP]['osmatch'][0]['name'],'vendor':nm[IP]['osmatch'][0]['osclass'][0]['vendor']}
        return nm[IP]['osmatch'][0]['osclass'][0]['vendor']
        
        
detect_snmp_or_os()
RESULT

SNMP scanning timeout. Doing OS detect...


{'snmp': None,
 'os': {'name': 'F5 BIG-IP Local Traffic Manager load balancer (TMOS 11.6)',
  'vendor': 'F5 Networks'},
 'isp': None,
 'location': None,
 'time': None}

In [115]:
import requests
from functools import reduce
MYIP = requests.get('https://api.bigdatacloud.net/data/client-ip').json()['ipString']

def ip_into_int(ip):
        return reduce(lambda x,y:(x<<8)+y,map(int,ip.split('.')))
def is_internal_ip(ip):
    ip = ip_into_int(ip)
    net_a = ip_into_int('10.255.255.255') >> 24
    net_b = ip_into_int('172.31.255.255') >> 20
    net_c = ip_into_int('192.168.255.255') >> 16
    return ip >> 24 == net_a or ip >>20 == net_b or ip >> 16 == net_c

def detect_isp_and_location():
    global RESULT

    if is_internal_ip(IP):
        search = requests.get(f'https://api.incolumitas.com/?q={MYIP}').json()
    else:
        search = requests.get(f'https://api.incolumitas.com/?q={IP}').json()
    isp_name = search['asn']['org']
    isp_country = search['asn']['country']

    location_country_code = search['location']['country_code']
    
    RESULT['isp'] = {'isp_name':isp_name,'isp_country':isp_country}
    RESULT['location'] = {'location_country_code':location_country_code}
    print(RESULT)

detect_isp_and_location()

{'snmp': None, 'os': {'name': 'F5 BIG-IP Local Traffic Manager load balancer (TMOS 11.6)', 'vendor': 'F5 Networks'}, 'isp': {'isp_name': 'Mobile Business Group', 'isp_country': 'tw'}, 'location': {'location_country_code': 'TW'}, 'time': None}


In [ ]:
def security_level_analysis():
    with open(SECURITY_FILE) as file_input:
        clean_network_list = json.load(file_input)
    # if DEBUG_MOD:
    #     print(clean_network_list)
    if isp_name in clean_network_list['isp']['clean']:
        if enterprise_name in clean_network_list['hdm']['clean']:
            return 3
        if enterprise_name in clean_network_list['hdm']['unclean']:
            return -1
        else:
            return 1
    elif isp_name in clean_network_list['isp']['unclean']:
        return -1
    else:
        if enterprise_name in clean_network_list['hdm']['clean']:
            return 2
        if enterprise_name in clean_network_list['hdm']['unclean']:
            return -1
        else:
            return 0

In [119]:
import time
import json
def save_result():
    RESULT['ip'] = IP
    if is_internal_ip(IP):
        with open('local_ip_record.json','r') as file:
            record : dict = json.load(file)
            file.close()
        RESULT['time'] = time.time()
        RESULT['src'] = MYIP
        record[IP] = RESULT
        with open('local_ip_record.json','w') as file:
            file.write(json.dumps(record))
            file.close()
    else:
        requests.post(f'http://144.24.84.156/node',data=RESULT)
save_result()

In [ ]:
def check_time():
    